In [1]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]
header = ['Color', 'Diameter', 'Label']

In [2]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [3]:
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [4]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [5]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [6]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [13]:
class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
        return f'Is %s %s %s ?' % (header[self.column], condition, str(self.value))

In [15]:
Question(1,3)

Is Diameter >= 3 ?

In [16]:
q = Question(0,'Green')
q

Is Color == Green ?

In [17]:
example = training_data[0]
q.match(example)

True

In [18]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [19]:
true_rows, false_rows = partition(training_data, Question(0,'Red'))
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [20]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [24]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl ** 2
    return impurity

In [25]:
no_mixing = [['Apple'], ['Apple']]
gini(no_mixing)

0.0

In [26]:
some_mixing = [['Apple'], ['Orange']]
gini(some_mixing)

0.5

In [27]:
lot_of_mixing = [['Apple'], ['Orange'], ['Grape'], ['Blueberry'], ['Grapefruit']]
gini(lot_of_mixing)

0.7999999999999998

In [28]:
def info_gain(left, right, current_uncertanty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertanty - p*gini(left) - (1-p)*gini(right)

In [29]:
current_uncertanty = gini(training_data)
current_uncertanty

0.6399999999999999

In [30]:
true_rows, false_rows = partition(training_data, Question(0,'Green'))
info_gain(true_rows, false_rows, current_uncertanty)

0.1399999999999999

In [31]:
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertanty)

0.37333333333333324

In [32]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertanty = gini(rows)
    n_features = len(rows[0]) - 1
    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertanty)
            if gain >= best_gain:
                best_gain = gain
                best_question = question
    return best_gain, best_question

In [33]:
best_gain, best_question = find_best_split(training_data)
best_question

Is Diameter >= 3 ?

In [34]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [35]:
class DecisionNode:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [36]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return DecisionNode(question, true_branch, false_branch)

In [40]:
def print_tree(node, spacing=''):
    if isinstance(node, Leaf):
        print(spacing + 'predict', node.predictions)
        return
    print(spacing + str(node.question))
    print(spacing + '-->True:')
    print_tree(node.true_branch, spacing + ' ')
    print(spacing + '-->False:')
    print_tree(node.false_branch, spacing + ' ')

In [41]:
my_tree = build_tree(training_data)

In [42]:
print_tree(my_tree)

Is Diameter >= 3 ?
-->True:
 Is Color == Yellow ?
 -->True:
  predict {'Apple': 1, 'Lemon': 1}
 -->False:
  predict {'Apple': 1}
-->False:
 predict {'Grape': 2}


In [43]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [44]:
classify(training_data[0], my_tree)

{'Apple': 1}

In [45]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + '%'
    return probs

In [46]:
print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [47]:
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [48]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]
for row in testing_data:
    print('Actual: %s. Predicted: %s' % (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
